In [7]:
!pip install opencv-python

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%%writefile aerial_data.yaml
train: /content/drive/MyDrive/DSCI_511_Project/aerial-cars-dataset/images
val: /content/drive/MyDrive/DSCI_511_Project/aerial-cars-dataset/images
nc: 1
names: ['car']

Overwriting aerial_data.yaml


In [3]:
!cat aerial_data.yaml

train: /content/drive/MyDrive/DSCI_511_Project/aerial-cars-dataset/images
val: /content/drive/MyDrive/DSCI_511_Project/aerial-cars-dataset/images
nc: 1
names: ['car']


In [4]:
import os

labels_dir = "/content/drive/MyDrive/DSCI_511_Project/aerial-cars-dataset/labels"
bad_files = []

for filename in os.listdir(labels_dir):
    if filename.lower().endswith('.txt'):
        file_path = os.path.join(labels_dir, filename)
        with open(file_path, "r") as f:
            lines = f.readlines()
        for line in lines:
            parts = line.strip().split()
            if parts and parts[0] != "0":
                bad_files.append((filename, line.strip()))

if bad_files:
    print("Found labels not remapped to 0:")
    for fname, line in bad_files:
        print(f"{fname}: {line}")
else:
    print("All label files have class 0.")

All label files have class 0.


## Training The Model
using aerial_data.yaml which contains the path for training and label files


In [12]:
%cd "/content/drive/MyDrive/DSCI_511_Project/yolov5_new"
!python train.py --img 640 --batch 16 --epochs 50 --data "/content/aerial_data.yaml" --weights yolov5s.pt

/content/drive/.shortcut-targets-by-id/1AS-UN1w4o3QpNZIHTPMkAHdSEMNGRZzn/DSCI_511_Project/yolov5_new
wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-03-13 23:58:59.028861: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741910339.051362    3778 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741910339.057925    3778 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B a

## Prediction using the trained model

In [9]:
import os
import json
import torch
import random
import cv2

# Load your custom YOLOv5 model from exp14's best.pt checkpoint
model = torch.hub.load('ultralytics/yolov5', 'custom',
                       path='/content/drive/MyDrive/DSCI_511_Project/yolov5_new/runs/train/exp14/weights/best.pt',
                       force_reload=True)
model.conf = 0.25  # Adjust confidence threshold if needed

# Define the folder containing images for inference (adjust path as needed)
image_folder = "/content/drive/MyDrive/image"

# Folder to store rendered images with detections
output_folder = "/content/drive/MyDrive/DSCI_511_Project/results/validation"
os.makedirs(output_folder, exist_ok=True)

results_list = []       # List of dicts for JSON output
detection_results = {}  # To store full result objects for visualization and saving

# Loop over images in the folder and run inference
for filename in os.listdir(image_folder):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(image_folder, filename)
        try:
            results = model(image_path)
        except FileNotFoundError:
            print(f"File not found, skipping: {filename}")
            continue
        detections = results.xyxy[0]  # [x1, y1, x2, y2, conf, class]
        car_count = detections.shape[0]  # each row corresponds to a detection
        results_list.append({
            "store_code": filename,
            "no of cars": int(car_count)
        })
        detection_results[filename] = results

        # Render the image with bounding boxes and save it
        rendered_imgs = results.render()  # returns list of rendered images
        for i, img in enumerate(rendered_imgs):
            # Construct output filename
            save_filename = f"validation_{filename}" if len(rendered_imgs)==1 else f"validation_{i}_{filename}"
            save_path = os.path.join(output_folder, save_filename)
            cv2.imwrite(save_path, img)

# Define output JSON path and save the results
json_path = "/content/drive/MyDrive/DSCI_511_Project/results/car_counts.json"
with open(json_path, "w") as f:
    json.dump(results_list, f, indent=4)

print("Results JSON saved to:", json_path)
print("Rendered images saved to:", output_folder)

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2025-3-14 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultra

Results JSON saved to: /content/drive/MyDrive/DSCI_511_Project/results/car_counts.json
Rendered images saved to: /content/drive/MyDrive/DSCI_511_Project/results/validation
